In [1]:
# %pip install cdsapi

import os
import cdsapi
from time import time

import xarray as xr
import matplotlib.pyplot as plt

In [2]:
c = cdsapi.Client(
    url="https://cds.climate.copernicus.eu/api/v2",
    key="54161:16b115ab-1a48-497f-82d0-d3d475a10993",
)  # Replace UID:ApiKey with you UID and Api Key

# define a logger
def print_to_file(*args, **kwargs):
    with open("era5_download.log", "a") as f:
        print(*args, file=f, **kwargs)

In [3]:
year = "2021"
master_init = time()
print_to_file("#"*20, f"Starting download for year {year}", "#"*20, sep="\n")
for month in ["01", "02", "03", "04", "05", "06", "07", "08","09","10","11","12"]:
    for hour in ["00:00", "01:00", "02:00", "03:00", "04:00", "05:00", "06:00", "07:00", "08:00", "09:00", "10:00", "11:00", "12:00", "13:00",  "14:00", "15:00", "16:00", "17:00", "18:00","19:00", "20:00", "21:00", "22:00", "23:00"]:
        init = time()
        c.retrieve(
            "reanalysis-era5-land",
            {
                "variable": [
                    "2m_temperature",
                    "skin_temperature",
                    "2m_dewpoint_temperature",
                    "snowfall",
                    "10m_u_component_of_wind",
                    "10m_v_component_of_wind",
                    "surface_pressure",
                    "total_precipitation",
                ],
                "year": year,
                "month": month,
                "day": [
                    "01","02","03","04","05","06","07","08","09","10","11","12","13","14","15","16","17","18","19","20","21","22","23","24","25","26","27","28","29","30","31",
                ],
                "time": hour,
                "area": [
                    42.00950826967187+0.2,
                    -124.48201686078049-0.2,
                    32.52883673637251-0.2,
                    -114.13122247508855+0.2,  # Bounding box for California
                ],
                "format": "netcdf",
            },
            f"/home/patel_zeel/aqmsp/aqmsp_data/data/era5/era5_{year}_{month}_{hour}.nc",
        )
        print_to_file(f"Downloaded era5_year_{year}_month_{month}_hour_{hour}.nc", end=" ")
        end = time()
        print_to_file(f"Time taken: {end-init} seconds")

master_end = time()
print_to_file(f"Total time taken: {(master_end-master_init)/60} minutes")

2023-09-21 00:45:45,257 INFO Welcome to the CDS
2023-09-21 00:45:45,258 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-land
2023-09-21 00:45:50,915 INFO Request is completed
2023-09-21 00:45:50,916 INFO Downloading https://download-0005-clone.copernicus-climate.eu/cache-compute-0005/cache/data1/adaptor.mars.internal-1695236703.9470966-9174-19-d79d2f5d-b6f5-490a-a9b2-9ced80b6d19d.nc to /home/patel_zeel/aqmsp/aqmsp_data/data/era5/era5_2021_01_00:00.nc (5.1M)
2023-09-21 00:45:53,038 INFO Download rate 2.4M/s
2023-09-21 00:45:53,381 INFO Welcome to the CDS
2023-09-21 00:45:53,382 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-land
2023-09-21 00:45:53,600 INFO Downloading https://download-0005-clone.copernicus-climate.eu/cache-compute-0005/cache/data8/adaptor.mars.internal-1695236793.0291367-11088-8-5e27d26c-0d8b-4526-8248-b910becd7b06.nc to /home/patel_zeel/aqmsp/aqmsp_data/data/era5/era5_2021_01_01:00.

In [4]:
era_ds = xr.open_dataset("/home/patel_zeel/aqmsp/aqmsp_data/data/era5/era5_2021_01_00:00.nc")
era_ds

<xarray.Dataset>
Dimensions:    (longitude: 108, latitude: 99, time: 31)
Coordinates:
  * longitude  (longitude) float32 -124.7 -124.6 -124.5 ... -114.2 -114.1 -114.0
  * latitude   (latitude) float32 42.13 42.03 41.93 41.83 ... 32.53 32.43 32.33
  * time       (time) datetime64[ns] 2021-01-01 2021-01-02 ... 2021-01-31
Data variables:
    t2m        (time, latitude, longitude) float32 ...
    skt        (time, latitude, longitude) float32 ...
    d2m        (time, latitude, longitude) float32 ...
    sf         (time, latitude, longitude) float32 ...
    u10        (time, latitude, longitude) float32 ...
    v10        (time, latitude, longitude) float32 ...
    sp         (time, latitude, longitude) float32 ...
    tp         (time, latitude, longitude) float32 ...
Attributes:
    Conventions:  CF-1.6
    history:      2023-09-20 19:05:04 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...

In [5]:
# %pip install git+https://github.com/scotthosking/get-station-data.git
# from get_station_data import ghcnd
# from get_station_data.util import nearest_stn

In [6]:
# lat = 38.13
# lon = -122.0
# stn_md = ghcnd.get_stn_metadata()
# stn_md

In [7]:
# my_stns = nearest_stn(stn_md, lon, lat, 10)
# my_stns

In [8]:
# df = ghcnd.get_data(my_stns, date_range=("2020-12-30", "2021-01-31"))

# df.head()

In [9]:
# t_data = df[df["element"] == "TOBS"][df["station"] == "US1CACC0046"]
# t_data.head()

In [10]:
# tmp_era = era_ds.sel(latitude=lat, longitude=lon, method="nearest")

# # resample to daily and plot
# fig, ax = plt.subplots(figsize=(12, 4))
# tmp_era["t2m"].values